In [2]:
import os, sys
from dotenv import load_dotenv

In [4]:
rpath = os.path.abspath('/home/user/Documents/10/w1/tellco')

if rpath not in sys.path:
    sys.path.insert(0, rpath)

In [6]:
load_dotenv()
database_uri = os.getenv("DATABASE_URI")

In [8]:
import utils.database_manager as db
import utils.data_manager as dataManager

import script.data_summarizer as data_summarizer


In [10]:
dataM = dataManager.DataManager(database_uri)

Database connection established successfully.


In [12]:
data = dataM.read_table_to_dataframe('xdr_data')

In [13]:
data_summarizer.summarize_data(data)

,Data Type,Missing Values,Unique Values,mean,std,min,25%,50%,75%,max
Bearer Id,float64,991,134708,1.013887e+19,2.893173e+18,6.917538e+18,7.349883e+18,7.349883e+18,1.304243e+19,1.318654e+19
Start,object,1,9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Start ms,float64,1,1000,4.991882e+02,2.886118e+02,0.000000e+00,2.500000e+02,4.990000e+02,7.490000e+02,9.990000e+02
End,object,1,6403,NaN,NaN,NaN,NaN,NaN,NaN,NaN
End ms,float64,1,1000,4.988009e+02,2.880977e+02,0.000000e+00,2.510000e+02,5.000000e+02,7.500000e+02,9.990000e+02
Dur. (ms),float64,1,89525,1.046086e+05,8.103762e+04,7.142000e+03,5.744050e+04,8.639900e+04,1.324302e+05,1.859336e+06
IMSI,float64,570,107265,2.082016e+14,2.148809e+10,2.040471e+14,2.082014e+14,2.082015e+14,2.082018e+14,2.140743e+14
MSISDN/Number,float64,1066,106856,4.188282e+10,2.447443e+12,3.360100e+10,3.365130e+10,3.366371e+10,3.368349e+10,8.823971e+14
IMEI,float64,572,107270,4.847455e+13,2.241637e+13,4.400152e+11,3.546071e+13,3.572201e+13,8.611970e+13,9.900120e+13
Last Location Name,object,1153,45547,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
data.shape

(150001, 55)

In [20]:
data_summarizer.categorical_summary(data)

,Unique Values,Value Counts
Start,9997,Start 4/26/2019 7:25 203 4/26/2019 7:26 ...
End,6403,End 4/25/2019 0:01 1150 4/25/2019 0:02 ...
Last Location Name,45547,Last Location Name D41377B ...
Handset Manufacturer,170,Handset Manufacturer Apple ...
Handset Type,1396,Handset Type Huawei B528S-23A ...


In [47]:
#top 3 handset manufacturers
data_summarizer.value_counts_summary(data,"Handset Type").head(10)

,Handset Type,count
0,Huawei B528S-23A,19752
1,Apple iPhone 6S (A1688),9419
2,Apple iPhone 6 (A1586),9023
3,undefined,8987
4,Apple iPhone 7 (A1778),6326
5,Apple iPhone Se (A1723),5187
6,Apple iPhone 8 (A1905),4993
7,Apple iPhone Xr (A2105),4568
8,Samsung Galaxy S8 (Sm-G950F),4520
9,Apple iPhone X (A1901),3813


In [49]:
#top 5 handsets per top 3 handset manufacturer

result = data_summarizer.value_counts_summary(data, 'Handset Type', display_columns=['Handset Manufacturer'])

# Filter the top 3 handset manufacturers
top_manufacturers = result['Handset Manufacturer'].value_counts().head(3).index.tolist()
top_handsets = result[result['Handset Manufacturer'].isin(top_manufacturers)]

# Group by Manufacturer and select the top 5 handsets
top_5_handsets = top_handsets.groupby('Handset Manufacturer').head(5)

print(top_5_handsets)


     Handset Type  count Handset Manufacturer
3       undefined   8987            undefined
4       undefined   8987            undefined
5       undefined   8987            undefined
6       undefined   8987            undefined
7       undefined   8987            undefined
9818    Test IMEI      1            Test IMEI


In [24]:
grouped_data = data.groupby('Bearer Id')
grouped_data

In [28]:
aggregation_columns = {
    'Bearer Id': 'count',  # Number of xDR sessions
    'Dur. (ms)': 'sum',  # Total session duration in milliseconds
    'Social Media DL (Bytes)': 'sum',  # Social media download data
    'Social Media UL (Bytes)': 'sum',  # Social media upload data
    'Youtube DL (Bytes)': 'sum',  # YouTube download data
    'Youtube UL (Bytes)': 'sum',  # YouTube upload data
    'Netflix DL (Bytes)': 'sum',  # Netflix download data
    'Netflix UL (Bytes)': 'sum',  # Netflix upload data
    'Google DL (Bytes)': 'sum',  # Google download data
    'Google UL (Bytes)': 'sum',  # Google upload data
    'Email DL (Bytes)': 'sum',  # Email download data
    'Email UL (Bytes)': 'sum',  # Email upload data
    'Gaming DL (Bytes)': 'sum',  # Gaming download data
    'Gaming UL (Bytes)': 'sum',  # Gaming upload data
    'Other DL (Bytes)': 'sum',  # Other download data
    'Other UL (Bytes)': 'sum'  # Other upload data
}


In [30]:
data_summarizer.group_and_aggregate(data,'Bearer Id',aggregation_columns)

,Bearer Id,Dur. (ms),Social Media DL (Bytes),Social Media UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes)
Bearer Id,,,,,,,,,,,,,,,,
6.917538e+18,1,24534.0,2404741.0,2410.0,6139644.0,2071526.0,19494278.0,14668354.0,5791591.0,2871336.0,782388.0,806920.0,466109357.0,5333340.0,670751043.0,15950724.0
6.917538e+18,1,21489.0,2478607.0,11936.0,10281221.0,18119976.0,19455048.0,10631652.0,3605446.0,2825198.0,446376.0,525108.0,673282567.0,2670856.0,501608458.0,3908870.0
6.917538e+18,1,27786.0,944612.0,2827.0,5385159.0,4295851.0,15755839.0,1300571.0,10373157.0,56392.0,128003.0,34038.0,821879090.0,8521398.0,472846860.0,1337849.0
6.917538e+18,1,15635.0,1817239.0,19827.0,12347020.0,11089528.0,2859358.0,1738176.0,269988.0,3696393.0,3191192.0,896670.0,805301713.0,16257481.0,24303797.0,15907613.0
6.917538e+18,1,24264.0,1867318.0,18928.0,15231815.0,8401567.0,21563985.0,2817981.0,1689296.0,195216.0,740633.0,590043.0,583864716.0,6992868.0,685122214.0,2966860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.318654e+19,1,80024.0,3240226.0,38284.0,12404964.0,4343114.0,11108134.0,21649273.0,2036152.0,2271168.0,2410615.0,387548.0,114093049.0,2834548.0,695881178.0,3888729.0
1.318654e+19,1,145291.0,3062671.0,48953.0,22147919.0,2152449.0,21468525.0,8603105.0,9363661.0,4001970.0,2192057.0,866373.0,328766801.0,7569327.0,371261255.0,12947410.0
1.318654e+19,1,86399.0,720996.0,42836.0,11879062.0,1290963.0,22596930.0,11943452.0,1541915.0,2100839.0,2315638.0,839789.0,833634251.0,10607174.0,697260277.0,9094407.0
